<a href="https://colab.research.google.com/github/saeidakbari3364/Deep-Learning/blob/main/DeepLearning365.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
mnist_dataset,mnist_info=tfds.load(name='mnist',with_info=True,as_supervised=True)
mnist_dataset

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


{'test': <_PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 'train': <_PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}

In [3]:
mnist_info.splits['train'].num_examples

60000

In [4]:
mnist_dataset['train']

<_PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [5]:
mnist_train,mnist_test=mnist_dataset['train'],mnist_dataset['test']
num_validation_samples=0.1*mnist_info.splits['train'].num_examples
num_validation_samples=tf.cast(num_validation_samples,tf.int64)

In [6]:
num_validation_samples

<tf.Tensor: shape=(), dtype=int64, numpy=6000>

In [7]:
num_test_samples=mnist_info.splits['test'].num_examples
num_test_samples=tf.cast(num_test_samples,tf.int64)

In [8]:
num_test_samples

<tf.Tensor: shape=(), dtype=int64, numpy=10000>

In [9]:
def scale(image,label):
  img=tf.cast(image,tf.float32)
  img /=255.
  return img,label

In [10]:
scaled_train_and_validation_data=mnist_train.map(scale)

In [11]:
scaled_train_and_validation_data

<_MapDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [12]:
test_data=mnist_test.map(scale)

In [13]:
BUFFER_SIZE=10000
shuffle_train_and_validation_data=scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
shuffle_train_and_validation_data

<_ShuffleDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [14]:
validation_data=shuffle_train_and_validation_data.take(num_validation_samples)

In [15]:
train_data=shuffle_train_and_validation_data.skip(num_validation_samples)

In [16]:
BATCH_SIZE=100
train_data=train_data.batch(BATCH_SIZE)

In [17]:
validation_data=validation_data.batch(num_validation_samples)

In [18]:
test_data=test_data.batch(num_test_samples)

In [19]:
validation_inputs,validation_targets=next(iter(validation_data))

In [20]:
input_size=784
output_size=10
hidden_layer_size=50


model=tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(output_size,activation='softmax')
    ])

In [21]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [22]:
NUM_EPOCHS=5

model.fit(train_data,epochs=NUM_EPOCHS,validation_data=(validation_inputs,validation_targets),verbose=2)

Epoch 1/5
540/540 - 15s - loss: 0.4278 - accuracy: 0.8804 - val_loss: 0.2130 - val_accuracy: 0.9400 - 15s/epoch - 27ms/step
Epoch 2/5
540/540 - 6s - loss: 0.1918 - accuracy: 0.9450 - val_loss: 0.1622 - val_accuracy: 0.9535 - 6s/epoch - 11ms/step
Epoch 3/5
540/540 - 8s - loss: 0.1469 - accuracy: 0.9574 - val_loss: 0.1254 - val_accuracy: 0.9640 - 8s/epoch - 15ms/step
Epoch 4/5
540/540 - 5s - loss: 0.1192 - accuracy: 0.9648 - val_loss: 0.1114 - val_accuracy: 0.9677 - 5s/epoch - 10ms/step
Epoch 5/5
540/540 - 7s - loss: 0.1018 - accuracy: 0.9691 - val_loss: 0.1035 - val_accuracy: 0.9707 - 7s/epoch - 13ms/step


In [23]:
test_loss,test_accuracy=model.evaluate(test_data)

1/1 [==============================] - 2s 2s/step - loss: 0.1160 - accuracy: 0.9641
